<a href="https://colab.research.google.com/github/GezhinOleg/Num_Pnd_L7/blob/main/Basics_parsing_and_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1.

Обязательная часть Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная) Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup as bs

In [4]:
KEYWORDS = {'python',  'парсинг'}
BASEURL = 'https://habr.com/ru/all/'

In [5]:
def get_url_text(url):
    response = requests.get(url)
    if response.ok:
        return response.text
    else:
        return False

In [6]:
def main(url=BASEURL):
    text = get_url_text(url)
    soup = bs(text, features='html.parser')
    articles = soup.find_all('article')
    article_list = []
    for article in articles:
        article_time = article.find('span', class_='tm-article-snippet__datetime-published').text
        # Получаем время написания статьи
        article_title = article.find('a', class_='tm-article-snippet__title-link')
        # Получаем текст лежащий между тегами <a></a>
        article_title_link = article_title['href']
        # Получаем текст сокращенной ссылки
        hubs = [h.text.strip() for h in article.find_all('a', class_='tm-article-snippet__title-link')]
        st_list = hubs[0].lower().split()
        # Буквы полученного текста приводим к строчному представлению, чтобы не упустить слова из-за написания
        for st in st_list:
            if st in KEYWORDS:
                text = ''.join(hubs)
                article_list.append(f"{article_time} - {text} - https://habr.com{article_title_link}")
    return article_list


In [7]:
if __name__ == '__main__':
    output_list = []
    pages = int(input('Введите количество страниц: '))
    for page in range(pages):
        print(f'Обрабатываем страницу {page + 1} из {pages}')
        if page == 0:
            output_list += main()
        else:
            output_list += main(url=f'{BASEURL}page{page+1}/')
    print(*output_list, sep='\n')

Введите количество страниц: 3
Обрабатываем страницу 1 из 3
Обрабатываем страницу 2 из 3
Обрабатываем страницу 3 из 3
20  августа   в 16:21 - predict_proba в Python не прогнозирует вероятности (и как с этим бороться) - https://habr.com/ru/company/otus/blog/573924/


## Задание 2.

Обязательная часть Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная) Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology’ TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [ ]:
import requests
import json

In [ ]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Accept': 'application/json, text/plain, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json;charset=UTF-8',
        'Host': 'identityprotection.avast.com',
        'Origin': 'https://www.avast.com',
        'Referer': 'https://www.avast.com/',
        'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
        'sec-ch-ua-mobile': '?0',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
        'Vaar-Header-App-Build-Version': '1.0.0',
        'Vaar-Header-App-Product': 'hackcheck-web-avast',
        'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
        'Vaar-Version': '0',
        'Access-Control-Allow-Credentials': 'true',
        'Access-Control-Allow-Methods': 'POST',
        'Access-Control-Allow-Origin': 'https://www.avast.com',
        'Access-Control-Expose-Headers': 'Vaar-Version, Vaar-Status',
        'Access-Control-Max-Age': '86400',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        'Server': 'nginx',
        'Vaar-Header-Execution-Chain-Id': '3224d669-242b-47c2-bcba-689443db864d',
        'Vaar-Status': '0',
        'Vaar-Version': '0',
        'Vary': 'Origin,Access-Control-Request-Method'
    }


In [ ]:
def load_data():
    email_input = input('Введите через пробел e-mail, и нажмите "Enter"').split()
    payload = json.dumps({'emailAddresses': email_input})
    s = requests.post(url, headers=headers, data=payload)
    with open('outfile.txt', 'w', encoding='utf-8') as out_file:
        out_file.write(s.text)


In [ ]:
def edit_data():
    with open('mail_list.txt', 'w', encoding='utf-8') as m_file:
        m_file.write('')
    with open('outfile.txt', 'r', encoding='utf-8') as f:
        file = f.readline()
        dict_file = json.loads(file)
        for i in dict_file['breaches']:
            leak_date = dict_file['breaches'][i]['publishDate']
            leak_description = dict_file['breaches'][i]['description']
            leak_source = dict_file['breaches'][i]['site']
            mail_str = (f'{leak_date} - {leak_source} - {leak_description}' + '\n')
            with open('mail_list.txt', 'a', encoding='utf-8') as m_file:
                m_file.write(mail_str)


In [ ]:
def main():
    call = input('Для проверки e-mail, введите 1, для вывода результата проверки введите 2: ')
    if call == '1':
        load_data()
    elif call == '2':
        edit_data()


if __name__ == '__main__':
        main()